# todos

## Environment



- [ ]create `contracts_df` from `create_contracts()`.
   * Columns : 
     * `start_port`
     * `ship_type`
     * `end_port`
     * `start_day` =  start_step + (steps needed for pick up == dm / 14 knots) ? probably 10 knots
     * `end_day` = start_day + max_distance in distance_matrix / 10 knots in steps
     * `value_in_$` = values_dict for each ship type
     * `assigned_ship` = [0,1,2] None, ship_1, ship_2 
- [ ] create `can_reach` function that calculates if a ship can make it on time for the contract
   * maybe implement `can_reach` with 10,12,14 knots? 



## Agent
- [ ] Action Space:
   * Choose contract Discrete 5 [0,1,2,3,4] choose zero, one, two, three or all contracts
   * For each contract: 
     * Choose ship Discrete 3 [0,1,2]
     * Choose speed Discrete 3 [10,12,14]
- [ ] 

In [1]:
import gym
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [10]:
# load data
data_dict = {
    "fleet_path":'data/fleet.csv',
    "ports_path":'data/ports_10.csv',
    "dm_path" : 'data/distance_matrix.csv'
}


ports_df = pd.read_csv('data/ports_10.csv')
distance_matrix = pd.read_csv('data/distance_matrix.csv')
fleet_df = pd.read_csv('data/fleet.csv')



In [12]:
# set fleet at random ports

fleet_df['port'] =  np.random.randint(1,11,fleet_df.shape[0])
fleet_df

,name,type,dwt,cii_threshold,cii,port,speed,is_available
0,handy_1,handymax,40000,0,0,6,0,1
1,handy_2,handymax,40000,0,0,8,0,1
2,supra_1,supramax,50000,0,0,4,0,1
3,supra_2,supramax,50000,0,0,8,0,1
4,ultra_1,ultramax,60000,0,0,7,0,1
5,ultra_2,ultramax,60000,0,0,10,0,1
6,pana_1,panamax,70000,0,0,5,0,1
7,pana_2,panamax,70000,0,0,8,0,1
8,kamsar_1,kamsarmax,80000,0,0,4,0,1
9,kamsar_2,kamsarmax,80000,0,0,4,0,1


In [122]:
distance_matrix

,Ports,Shangai,Singapore,Xiamen,Dalian,Guangzhou,Qingdao,Busan,Hong Kong,Tianjin,Kaohsiung
0,Shangai,0,2237,585,551,928,399,492,845,696,600
1,Singapore,2237,0,1653,2619,1543,2463,2503,1460,2764,1621
2,Xiamen,585,1653,0,1002,370,846,886,287,1147,124
3,Dalian,551,2619,1002,0,1335,356,543,1256,198,1029
4,Guangzhou,928,1543,370,1335,0,1183,1223,83,1480,425
5,Qingdao,399,2463,846,356,1183,0,502,1100,412,876
6,Busan,492,2503,886,543,1223,502,0,1140,688,908
7,Hong Kong,845,1460,287,1256,83,1100,1140,0,287,342
8,Tianjin,696,2764,1147,198,1480,412,688,287,0,1174
9,Kaohsiung,600,1621,124,1029,425,876,908,342,1174,0


In [174]:
dm_simple = distance_matrix.iloc[:,1:]
dm_simple 

,Shangai,Singapore,Xiamen,Dalian,Guangzhou,Qingdao,Busan,Hong Kong,Tianjin,Kaohsiung
0,0,2237,585,551,928,399,492,845,696,600
1,2237,0,1653,2619,1543,2463,2503,1460,2764,1621
2,585,1653,0,1002,370,846,886,287,1147,124
3,551,2619,1002,0,1335,356,543,1256,198,1029
4,928,1543,370,1335,0,1183,1223,83,1480,425
5,399,2463,846,356,1183,0,502,1100,412,876
6,492,2503,886,543,1223,502,0,1140,688,908
7,845,1460,287,1256,83,1100,1140,0,287,342
8,696,2764,1147,198,1480,412,688,287,0,1174
9,600,1621,124,1029,425,876,908,342,1174,0


In [140]:
dist_cols = distance_matrix.columns.to_list()
del dist_cols[0]
dist_cols 
dm_array = distance_matrix.loc[:,dist_cols].to_numpy()

array([[   0, 2237,  585,  551,  928,  399,  492,  845,  696,  600],
       [2237,    0, 1653, 2619, 1543, 2463, 2503, 1460, 2764, 1621],
       [ 585, 1653,    0, 1002,  370,  846,  886,  287, 1147,  124],
       [ 551, 2619, 1002,    0, 1335,  356,  543, 1256,  198, 1029],
       [ 928, 1543,  370, 1335,    0, 1183, 1223,   83, 1480,  425],
       [ 399, 2463,  846,  356, 1183,    0,  502, 1100,  412,  876],
       [ 492, 2503,  886,  543, 1223,  502,    0, 1140,  688,  908],
       [ 845, 1460,  287, 1256,   83, 1100, 1140,    0,  287,  342],
       [ 696, 2764, 1147,  198, 1480,  412,  688,  287,    0, 1174],
       [ 600, 1621,  124, 1029,  425,  876,  908,  342, 1174,    0]])

In [123]:
ports_df = ports_df.loc[:,['number','name','country']]
ports_df

,number,name,country
0,1,Shanghai,China
1,2,Singapore,Singapore
2,3,Xiamen,China
3,4,Dalian,China
4,5,Guangzhou,China
5,6,Qingdao,China
6,7,Busan,South Korea
7,8,Hong Kong,Hong Kong
8,9,Tianjin,China
9,10,Kaohsiung,Taiwan


In [125]:
ports_df.loc[0]

number            1
name       Shanghai
country       China
Name: 0, dtype: object

In [ ]:
# df_contr_total = start_port, ship_type, end_port, start_day, end_day, value, assigned_ship
# 4*365 = 1460 rows
df_contr_total = pd.DataFrame(columns=['start_port_number','end_port_number','contract_type','start_day','end_day','value','cargo_size','contract_duration','port_distance','assigned_ship'],
                            index = range(1,1461))
df_contr_total

# we will append df_contr_per_day to contracts 

In [239]:
df_contr_per_day = pd.DataFrame(columns=['start_port_number','end_port_number','contract_type','start_day','end_day','value','cargo_size','contract_duration','port_distance','assigned_ship'],
                            index = range(0,4))

contracts_df = df_contr_per_day.copy()

In [240]:
# create random contracts
def create_contrs(self=None, df=contracts_df, dm = distance_matrix , step=1, seed=None):
        """
        A method that creates 4 random cargo contracts at each env step. \ 
        There are 5 cargo contract types handy,supra,ultra,pana,kamsar for each port.\ 
        The function :
        * returns a 1d array of 4 elements
        * all array elements are integers from 0 to 4
        * the sum of the array must be 4
        """
        day = step
        # creating 4 contracts for the specific day
        contr_count = 4
        np.random.seed(seed)
        # x array of 5 elements that sum to one
        x = np.random.multinomial(contr_count, [1/5]*5, None)
        # y array of 45 zeros
        z = np.zeros(shape=45, dtype=np.int32)
        # contracts of size 50 concatenation of x,y
        contracts = np.concatenate((x,z))
        np.random.shuffle(contracts)
        
        # getting port locations where contracts have spawned 
        locations = np.nonzero(contracts)[0]
        # getting the start ports of the contract 
        ports = np.floor_divide(locations,5)
        # ports start at port 1
        ports = ports + 1
        # getting the contract count for a specific location
        contract_count_per_loc = contracts[locations]
        # array of fives to help with modulo
        fives = np.ones((len(locations)))*5
        # find ship type at port with modulo
        x = np.mod(locations,fives)
        # creating the ship_conditions for each shiptype
        ship_conditions = [ x == 0 , x == 1, x == 2, x == 3, x == 4 ]
        ship_choices = ['handymax','supramax','ultramax','panamax','kamsarmax']
        # getting the shiptype and converting it to str
        ship_types = np.select(ship_conditions,ship_choices)
        ship_types = ship_types.astype(str)
        # populating the df
        df['contract_type'] = np.repeat(ship_types,contract_count_per_loc)
        df['start_port_number'] = np.repeat(ports,contract_count_per_loc)
        df['end_port_number'] = np.random.randint(low=1, high=11, size=(4,))
        # Check for contracts with same start and end ports
        same_ports = df['start_port_number'] == df['end_port_number']
        # repeat until no start and end ports of the same contract are the same 
        while sum(same_ports) != 0 :
          df['end_port_number'] = np.where(same_ports, np.random.randint(low=1, high=11, size=same_ports.shape), df['end_port_number'])
          same_ports = df['start_port_number'] == df['end_port_number']

        # get distance between start and end ports arrays
        start_port_numbers_index = df['start_port_number'] - 1
        
        end_port_numbers_index = df['end_port_number']
        
        dist_df = dm.iloc[start_port_numbers_index,end_port_numbers_index]
        # the distance 
        df['port_distance'] = pd.Series(np.diag(dist_df)).reindex()


        # Create cargo size based on ship_type
        type_conditions = [ df['contract_type'] == 'handymax', 
                            df['contract_type'] == 'supramax',
                            df['contract_type'] == 'ultramax',
                            df['contract_type'] == 'panamax',
                            df['contract_type'] == 'kamsarmax']

        cargo_size_choices = [np.random.randint(30_000,40_000,type_conditions[0].shape),
                              np.random.randint(40_000,50_000,type_conditions[1].shape),
                              np.random.randint(50_000,60_000,type_conditions[2].shape),
                              np.random.randint(60_000,70_000,type_conditions[3].shape),
                              np.random.randint(70_000,80_000,type_conditions[4].shape)]

        df['cargo_size'] = np.select(type_conditions,cargo_size_choices)
        
        # add contract value 
        df['value'] = np.round(df['cargo_size'] * 1.5)

        # add start day
        df['start_day'] = day
         


        # calculate duration

        # pick random speed from possible set of speeds
        u_picked = np.random.choice([10,12,14])
        
        # pick distance between ports from df
        dx = df['port_distance']
        # find duration of trip between ports with picked speed in hours
        dt_hours = ( dx / u_picked).round()
        # find duration of trip between ports in days
        dt_days = (dt_hours / 24).round()

        # get upper triangle entries of distance matrix
        x = dm.iloc[:,1:].to_numpy(dtype=np.int32)
        mask_upper = np.triu_indices_from(x,k=1)
        triu = x[mask_upper]
        # average voyage distance between ports in the distance matrix
        avg_dx = np.round(triu.mean())
        # average voyage duration between ports with picked speed in hours
        avg_dt_hours = np.round(avg_dx/u_picked)
        # # average voyage duration between ports with picked speed in days
        avg_dt_days = np.round(avg_dt_hours / 24)
        
        # total duration
        df['contract_duration'] = dt_days + avg_dt_days


        # end_day ends at 23:59
        df['end_day'] = df['start_day'] + df['contract_duration'] - 1
        
        return df
        


In [242]:
contracts = create_contrs(seed=1)
contracts

,start_port_number,end_port_number,contract_type,start_day,end_day,value,cargo_size,contract_duration,port_distance,assigned_ship
0,3,5,supramax,1,5.0,60232.0,40155,5.0,370,NaN
1,7,1,panamax,1,6.0,90364.0,60243,6.0,492,NaN
2,10,4,supramax,1,8.0,60972.0,40648,8.0,1029,NaN
3,10,4,panamax,1,8.0,91821.0,61214,8.0,1029,NaN


In [224]:
contracts.loc[0]

start_port_number           3
end_port_number             5
contract_type        supramax
start_day                   1
end_day                   5.0
value                 60232.0
cargo_size              40155
duration                  5.0
port_distance             370
assigned_ship             NaN
Name: 0, dtype: object

In [247]:
# Delay

contracts = create_contrs(seed=56)
agent_speed = 10
agent_ship = 2
ship_current_port = 4
contract = contracts.loc[0]
start_port_number = contract.start_port_number
end_port_number = contract.end_port_number
trip_1_hours = find_distance(ship_current_port,start_port_number) / agent_speed
print(dt_1_hours)
trip_2_hours = find_distance(start_port_number,end_port_number) / agent_speed
print(dt_2_hours)
trip_total_hours = trip_1_hours + trip_2_hours


trip_total_days = round(trip_total_hours / 24)
print(trip_total_days)

contract_duration = contract.contract_duration
delay = trip_total_days - contract_duration
delay_penalty_factor_per_day = 10

reward = 0
penalty = 0 
if delay <= 0:
    # I arrived on time
    # provide a constant reward > 0 
    reward = 10
    # 
else:
    # I arrived late 
    # penalty < 0
    delay_in_days = -delay
    penalty = delay_penalty_factor_per_day * delay_in_days

print(f"The delay is {delay} days")
print(f"The reward is {reward}")
print(f"The penalty is {penalty}")

100.2
37.0
2
The delay is -4.0 days
The reward is 10
The penalty is 0


In [232]:
find_distance(ship_current_port,start_port_number)

1002

In [182]:
def find_distance(port_number_1, port_number_2,dist_m=distance_matrix):
    """
    find port distances from port numbers
    port numbers are port are port indices + 1
    port numbers must be in [1-10] range
    """
    dist_m = dist_m.iloc[:,1:]
    idx_1 = port_number_1-1
    idx_2 = port_number_2-1
    distance = dist_m.iloc[idx_1,idx_2]
    return distance

In [105]:
import gym 
import pandas as pd
import numpy as np
from gym import spaces
from utils.utils import cii_expected

class CarbonEnv(gym.Env):
    """
    Description :

        A custom openai gym environment for the carbon emission problem.

    """


    def __init__(self,data_dict):
        super().__init__()

        NUM_CONTRACTS_PER_STEP = 4
        NUM_SHIPS = 10
        SET_OF_SPEEDS = [10,12,14]
        MIN_REVENUE = 1_000_000
       # TERMINATION_FROM_CII = # AT LEAST 5 SHIPS DROP THEIR RATING

        self.fleet = pd.read_csv(data_dict['fleet_path'])
        self.ports = pd.read_csv(data_dict['ports_path'])
        self.dm = pd.read_csv(data_dict['dm_path'])

        # calculate fleet's required cii
        self.fleet['cii_required'] = self.fleet['dwt'].map(cii_expected)

        # set fleet at random ports
        self.fleet['current_port'] = np.random.randint(1,11,self.fleet.shape[0])

        self.state = None
        self.previous_action = None



        observation_space = spaces.Dict({
            "daily_contracts" : spaces.Discrete(NUM_CONTRACTS_PER_STEP,), # 4 new contracts each day
            "available_ships" : spaces.Box(low=0,high=1,shape=(NUM_SHIPS,),dtype=np.int32) # [0,1,1,0,0,1,0,1,1,0]
                                                                                            # ships: 1,2,5,7,8 are available
        })

        action_space = spaces.Dict({
            "choose_contract" : spaces.Box(low=0,high=1,shape=(4,),dtype=np.int32), # (0,0,1,1) deny contracts 0,1 accept contracts 2,3
            "choose_ship" : spaces.Box(low=0, high=10, shape=(4,),dtype=np.int32), # (0,0,5,7) assign ships 5,7 to contracts 2,3
            "choose_speed" : spaces.Box(low=0, high=3, shape=(4,),dtype=np.int32) # (0,0,1,2) 10 knots for ship 5, 12knots for ship 7  
            })


        self.reset()

    def create_contracts(self, seed=None):
        """
        A method that creates 4 random cargo contracts at each env step. \ 
        There are 5 cargo contract types handy,supra,ultra,pana,kamsar for each port.\ 
        The function :
        * returns a 1d array of 50 elements
        * all array elements are integers from 0 to 4
        * the sum of the array must be 4
        """
        contract_count = 4
        np.random.seed(seed)
        x = np.random.multinomial(contract_count, [1/5]*5, None)
        z = np.zeros(shape=45, dtype=np.int32)
        c = np.concatenate((x,z))
        np.random.shuffle(c)
        return c




    def step(self, action):
        # Execute one time step in the environment based on the action chosen by the model
        # action will have shape 3x4
        print(len(action))
        print(action)
        pass

    def reset(self):
        # Set the state of the environment to an initial state
        # return self._RESET()
        pass

    def _STEP(self,action_dict):
        pass
        
    def _RESET(self):
        #return self.state
        pass





    pass




In [39]:
envo = CarbonEnv(data_dict)

In [ ]:
# training loop schema

# agent = GraphAttentionModel


# years = 3000 
# for i in range(years): 
#     obs = env.reset()
#     done = False 
#     score = 0
#     day = 1
#     while not done: # while year is not done
#         act = agent.choose_action(obs)
#         new_state, reward, done, info = env.step(act)
#         agent.remember(obs,act, reward, new_state, int(done))
#         agent.learn()
#         score += reward
#         day += 1
#         obs = new_state 

In [105]:
def get_ship_type(contrs):
    pass
